In [ ]:
#Code for FL_Zone_9a_9b_Scrub_Incursion_Restoratoin Project 
#Data Setup
#GEOG561
#Mike Karasoff

import arcpy
import numpy as np
DBPATH=r"C:\Users\mike\Documents\OSU\GEOG561\Project - Florida Scrub Zone\FL_Zone_9a_9b_Scrub_Incursion_Restoration\FL_Zone_9a_9b_Scrub_Incursion_Restoration.gdb"
DATA_PATH=r"C:\Users\mike\Documents\OSU\GEOG561\Project - Florida Scrub Zone\Data"

arcpy.env.workspace=DBPATH




In [ ]:
#Create a feature set from the community plants in zone9

#Start with the community plant data
arcpy.management.XYTableToPoint("scrubCommunityPlants.csv", "csScrubCommunityPlants", "decimalLongitude", "decimalLatitude")

In [ ]:
#Pare down the table.  Lots of unused fields
FIELDS_TO_KEEP=['OBJECTID', 'Shape', 'gbifID', 'coordinateUncertaintyInMeters', 
                'decimalLongitude', 'decimalLatitude', 'species', 'elevation']
fields = [f.name for f in arcpy.ListFields('csScrubCommunityPlants')]
fieldsRemove = set(fields.copy())
fieldsRemove = fieldsRemove - set(FIELDS_TO_KEEP)

cursor = arcpy.da.SearchCursor('csScrubCommunityPlants', fields)
for row in cursor:
    for i in range(len(fields)):
        if row[i] is not None:
            fieldsRemove = fieldsRemove - set((fields[i],))
            
print("Fields to remove:", fieldsRemove)
arcpy.management.DeleteField('csScrubCommunityPlants', list(fieldsRemove))

In [ ]:
#Combine zone 9a and zone 9b to make "zone 9"
zone9_sel = arcpy.management.SelectLayerByAttribute(
    r"C:\Users\mike\Documents\OSU\GEOG561\Project - Florida Scrub Zone\Data\USDA_Zones\phm_us_shp.shp",  
    where_clause = "ZONE = '9a: 20 to 25 F' Or ZONE = '9b: 25 to 30 F'")

#We will want to enumerate the plant types for further analysis
plants_enum = {}

arcpy.management.CopyFeatures(zone9_sel, 'zone9')

In [ ]:
#Select plants in zone 9
csScrubPlantsZone9_sel=arcpy.management.SelectLayerByLocation("csScrubCommunityPlants", 
                                                                     "INTERSECT", 
                                                                     "zone9", "", 
                                                                     "NEW_SELECTION")

#Filter points with large or <Null> uncertainty 
csScrubPlantsZone9 = arcpy.management.MakeFeatureLayer(csScrubPlantsZone9_sel, 
                    "csScrubPlantsZone9", 
                    where_clause = "coordinateUncertaintyInMeters < 150 Or coordinateUncertaintyInMeters IS NULL") 

arcpy.management.CopyFeatures(csScrubPlantsZone9, 'csScrubPlantsZone9')

In [ ]:
#Cluster Point Data as Communities
# This must set the output Spatial Reference. 
# The factory code for NAD_1983_2011_StatePlane_Florida_East_FIPS_0901 is 6437.
with arcpy.EnvManager(outputCoordinateSystem=arcpy.SpatialReference(6437)):
    arcpy.gapro.FindPointClusters(csScrubPlantsZone9, 
                                  "csScrubPlantsZone9Clusters", 
                                  "DBSCAN", 30, "600 Meters")
    
#Remove points that were classified "noise" by cluster
csScrubPlantsZone9Communities = arcpy.management.SelectLayerByAttribute("csScrubPlantsZone9Clusters", 
                                             where_clause = "CLUSTER_ID <> -1")
arcpy.management.CopyFeatures(csScrubPlantsZone9Communities, "csScrubPlantsZone9Communities")
arcpy.management.Delete("csScrubPlantsZone9Clusters")


In [ ]:
#Create Polygons from the point clusters to estimate areas
with arcpy.EnvManager(outputCoordinateSystem=arcpy.SpatialReference(6437)):
    arcpy.management.MinimumBoundingGeometry("csScrubPlantsZone9Communities", 
                                             "csScrubPlantsZone9Communities_poly", 
                                             "CONVEX_HULL", "LIST", "CLUSTER_ID", "MBG_FIELDS")

In [ ]:
#Get known zone9 scrub lands from CLC

arcpy.management.MakeFeatureLayer(
    r"C:\Users\mike\Documents\OSU\GEOG561\Project - Florida Scrub Zone\Data\FL_Classified_Land_Cover.gdb\CLC_v3_4_poly", 
    "knownScrub", "NAME_SITE = 'Scrub'", None)

knownScrubZone9=arcpy.management.SelectLayerByLocation("knownScrub", 
                                                        "INTERSECT", 
                                                        "zone9", "", 
                                                        "NEW_SELECTION")


arcpy.management.CopyFeatures('knownScrub', 'knownScrub')
arcpy.management.CopyFeatures(knownScrubZone9, 'knownScrubZone9')

arcpy.management.Delete('knownScrub')
arcpy.management.Delete('knownScrub')


In [ ]:
#Get CS observations from known zone9 scrub lands from CLC

csKnownScrubZone9=arcpy.management.SelectLayerByLocation(csScrubPlantsZone9, 
                                                        "INTERSECT", 
                                                        knownScrubZone9, 
                                                        "250 Meters", 
                                                        "NEW_SELECTION")

arcpy.management.CopyFeatures(csKnownScrubZone9, 'csKnownScrubZone9')


In [ ]:
with arcpy.EnvManager(outputCoordinateSystem="PROJCS['Albers Conical Equal Area [Florida Geographic Data Library]',GEOGCS['GCS_North_American_1983_HARN',DATUM['D_North_American_1983_HARN',SPHEROID['GRS_1980',6378137.0,298.257222101]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Albers'],PARAMETER['False_Easting',400000.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',-84.0],PARAMETER['Standard_Parallel_1',24.0],PARAMETER['Standard_Parallel_2',31.5],PARAMETER['Central_Parallel',24.0],UNIT['Meter',1.0]]", rasterStatistics="NONE", snapRaster="rfExtentRaster", parallelProcessingFactor="75%", pyramid="NONE", 
                      extent="112328.6536 334500.8659 697135.871199999 733658.6742", nodata="MAP_UP", cellSize=r"C:\Users\mike\Documents\OSU\GEOG561\Project - Florida Scrub Zone\FL_Zone_9a_9b_Scrub_Incursion_Restoration\FL_Zone_9a_9b_Scrub_Incursion_Restoration.gdb\rfExtentRaster", geographicTransformations=None):
    arcpy.management.CopyRaster("wetlands_v44", r"C:\Users\mike\Documents\OSU\GEOG561\Project - Florida Scrub Zone\FL_Zone_9a_9b_Scrub_Incursion_Restoration\FL_Zone_9a_9b_Scrub_Incursion_Restoration.gdb\wetlands", '', None, "15", "NONE", "NONE", "4_BIT", "NONE", "NONE", "JPEG", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")